# **End-To-End Deployment of Restaurant Ratings**

# **ABSTRACT**

Zomato is one of the best online food delivery apps which gives the users the ratings and the reviews on restaurants all over india.These ratings and the Reviews are considered as one of the most important deciding factors which determine how good a restaurant is. 

We will therefore use the real time Data set with variuos features a user would look into regarding a restaurant. We will be considering Banglore City in this analysis.

Content
The basic idea of analyzing the Zomato dataset is to get a fair idea about the factors affecting the establishment
of different types of restaurant at different places in Bengaluru, aggregate rating of each restaurant, Bengaluru
being one such city has more than 12,000 restaurants with restaurants serving dishes from all over the world.

With each day new restaurants opening the industry has’nt been saturated yet and the demand is increasing
day by day. Inspite of increasing demand it however has become difficult for new restaurants to compete with
established restaurants. Most of them serving the same food. Bengaluru being an IT capital of India. Most of the people here are dependent mainly on the restaurant food as they don’t have time to cook for themselves.

With such an overwhelming demand of restaurants it has therefore become important to study the demography
of a location. What kind of a food is more popular in a locality. Do the entire locality loves vegetarian food.
If yes then is that locality populated by a particular sect of people for eg. Jain, Marwaris, Gujaratis who are
mostly vegetarian. These kind of analysis can be done using the data, by studying the factors such as

    • Location of the restaurant
    • Approx Price of food
    • Theme based restaurant or not
    • Which locality of that city serves that cuisines with maximum number of restaurants
    • The needs of people who are striving to get the best cuisine of the neighborhood
    • Is a particular neighborhood famous for its own kind of food.

“Just so that you have a good meal the next time you step out”

The data is accurate to that available on the zomato website until 15 March 2019.
The data was scraped from Zomato in two phase. After going through the structure of the website I found that for each neighborhood there are 6-7 category of restaurants viz. Buffet, Cafes, Delivery, Desserts, Dine-out, Drinks & nightlife, Pubs and bars.

Phase I,

In Phase I of extraction only the URL, name and address of the restaurant were extracted which were visible on the front page. The URl's for each of the restaurants on the zomato were recorded in the csv file so that later the data can be extracted individually for each restaurant. This made the extraction process easier and reduced the extra load on my machine. The data for each neighborhood and each category can be found here

Phase II,

In Phase II the recorded data for each restaurant and each category was read and data for each restaurant was scraped individually. 15 variables were scraped in this phase. For each of the neighborhood and for each category their onlineorder, booktable, rate, votes, phone, location, resttype, dishliked, cuisines, approxcost(for two people), reviewslist, menu_item was extracted. See section 5 for more details about the variables.

Acknowledgements
The data scraped was entirely for educational purposes only. Note that I don’t claim any copyright for the data. All copyrights for the data is owned by Zomato Media Pvt. Ltd..

            Source: Kaggle

**Main Objective:**

The main agenda of this project is:

>> Perform extensive **Exploratory Data Analysis(EDA)** on the Zomato Dataset.

>>Build an appropriate **Machine Learning Model** that will help various Zomato Restaurants to predict their respective Ratings based on certain features

>>**DEPLOY** the Machine learning model via **Flask** that can be used to make live predictions of restaurants ratings

A step by step guide is attached to this documnet as well as a video explanation of each concpet.

In [1]:

#IMPORT THE NECESSARY LIBRARIES

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
import seaborn as sns

import matplotlib.ticker as mtick
plt.style.use('fivethirtyeight')
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import  ExtraTreesRegressor
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

**LOAD DATASET**

In [2]:

data = pd.read_csv('zomato.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'zomato.csv'

In [ ]:
data.head()

## Columns description

1. <b>url </B> contains the url of the restaurant in the zomato website

2. **address** contains the address of the restaurant in Bengaluru

3. **name** contains the name of the restaurant

4. **online_order** whether online ordering is available in the restaurant or not

5. **book_table** table book option available or not

6. **rate** contains the overall rating of the restaurant out of 5

7. **votes** contains total number of rating for the restaurant as of the above mentioned date

8. **phone** contains the phone number of the restaurant

9. **location** contains the neighborhood in which the restaurant is located

10. **rest_type** restaurant type

11. **dish_liked** dishes people liked in the restaurant

12. **cuisines** food styles, separated by comma

13. **approx_cost**(for two people) contains the approximate cost of meal for two people

14. **reviews_list** list of tuples containing reviews for the restaurant, each tuple

15. **menu_item** contains list of menus available in the restaurant

16. **listed_in**(type) type of meal

17. **listed_in**(**city**) contains the neighborhood in which the restaurant is listed

In [ ]:
data.shape

In [ ]:
data.dtypes #checking the data types

In [ ]:
data.isna().sum() #Checking null values

In [ ]:
#You can use pandas profiling to get an over all overview of the dataset
# import pandas_profiling as pf

# pf.ProfileReport(df)

In [ ]:
#Deleting Unnnecessary Columns
df=data.drop(['url','phone'],axis=1) #Dropping the column like "phone" and "url" and saving the new dataset as "df"

**Checking for duplicate values**

In [ ]:
df.duplicated().sum() 

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.duplicated().sum()

**Drop null values**

In [ ]:
#Remove the NaN values from the dataset
df.dropna(how='any',inplace=True)
df.isnull().sum()

**Renaming columes appropriately**

In [ ]:
df.columns

In [ ]:
df = df.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type',
                                  'listed_in(city)':'city'})
df.columns

In [ ]:
df.head()

**Cleaning the dataset**

In [ ]:
df['cost'].unique()

**replacing the "," with nothing and converting the results to float**

In [ ]:
#zomato['cost'] = zomato['cost'].astype(str) #Changing the cost to string
df['cost'] = df['cost'].apply(lambda x: x.replace(',','')) #Using lambda function to replace ',' from cost
df['cost'] = df['cost'].astype(float)

In [ ]:
print(df['cost'].unique())

print('---'*10)

df.dtypes

In [ ]:
#Reading uninque values from the Rate column
df['rate'].unique()

In [ ]:
df = df.loc[df.rate !='NEW'] #getting rid of "NEW"

In [ ]:
df['rate'].unique()

In [ ]:
#Removing '/5' from Rates

df['rate'] = df['rate'].apply(lambda x: x.replace('/5',''))



<br>

# Visualisations

Most famous restaurants chains in Bangaluru

In [ ]:
plt.figure(figsize=(17,10))
chains=df['name'].value_counts()[:20]
sns.barplot(x=chains,y=chains.index,palette='deep')
plt.title("Most famous restaurants chains in Bangaluru")
plt.xlabel("Number of outlets")
plt.show()

<br>

Whether restaurant offer Table booking or not

In [ ]:
x=df['book_table'].value_counts()
colors = ['#800080', '#0000A0']

trace=go.Pie(labels=x.index,values=x,textinfo="value",
            marker=dict(colors=colors, 
                           line=dict(color='#001000', width=2)))
layout=go.Layout(title="Table booking",width=600,height=600)
fig=go.Figure(data=[trace],layout=layout)
py.iplot(fig, filename='pie_chart_subplots')

**Insight**

Most of the Restaurants do not offer table booking

<br>
<br>

Whether Restaurants deliver online or Not

In [ ]:
#Restaurants delivering Online or not
sns.countplot(df['online_order'])
fig = plt.gcf()
fig.set_size_inches(10,10)
plt.title('Whether Restaurants deliver online or Not')
plt.show()

**Insight:**

Most Restaurants offer option for online order and delivery


<br>

<br>

## **Rating Distributions**

In [ ]:
#How ratings are distributed
plt.figure(figsize=(9,7))

sns.distplot(df['rate'],bins=20)



*  **Insight**

  We can infer from above that most of the ratings are within 3.5 and 4.5



In [ ]:
# #Distribution of the cost Vs ratings in parallel with online order
# plt.figure(figsize=(10,7))
# sns.scatterplot(x="rate",y='cost',hue='online_order',data=df)
# plt.show()

**Count of ratings as between "1 and 2", "2 and 3", "3 and 4", and "4 and 5"** 

In [ ]:
df['rate'].unique()

In [ ]:
df['rate'].min()

In [ ]:
df['rate'].max()

In [ ]:
df['rate']=df['rate'].astype(float)

In [ ]:
((df['rate']>=1) & (df['rate']<2)).sum()

In [ ]:
((df['rate']>=2) & (df['rate']<3)).sum()

In [ ]:
((df['rate']>=3) & (df['rate']<4)).sum()

In [ ]:
(df['rate']>=4).sum()

<br>

**Plotting the counts with the help of pie chart**

In [ ]:
slices=[((df['rate']>=1) & (df['rate']<2)).sum(),
        ((df['rate']>=2) & (df['rate']<3)).sum(),
        ((df['rate']>=3) & (df['rate']<4)).sum(),
        (df['rate']>=4).sum()
        ]

labels=['1<rate<2','2<rate<3','3<rate<4','>4']
colors = ['#ff3333','#c2c2d6','#6699ff']
plt.pie(slices,colors=colors, labels=labels, autopct='%1.0f%%', pctdistance=.5, labeldistance=1.2,shadow=True)
fig = plt.gcf()
plt.title("Percentage of Restaurants according to their ratings")

fig.set_size_inches(10,10)
plt.show()

<br>
<br>

## **Services Types**

In [ ]:
#Types of Services

sns.countplot(df['type']).set_xticklabels(sns.countplot(df['type']).get_xticklabels(), rotation=90, ha="right")
fig = plt.gcf()
fig.set_size_inches(12,12)
plt.title('Type of Service')

Here the two main service types are **Delivery** and **Dine-out**

<br>
<br>

## **Distribution of Cost of Food for two People**

In [ ]:
from plotly.offline import iplot

In [ ]:
trace0=go.Box(y=df['cost'],name="accepting online orders",
              marker = dict(
        color = 'rgb(113, 10, 100)',
    ))
data=[trace0]
layout=go.Layout(title="Box plot of approximate cost",width=800,height=800,yaxis=dict(title="Price"))
fig=go.Figure(data=data,layout=layout)
py.iplot(fig)

**Distribution of charges**

In [ ]:
plt.figure(figsize=(8,8))
sns.distplot(df['cost'])
plt.show()

## **Most Liked Dishes**

In [ ]:
#re=regular expression (use for splitting words)

import re

df.index=range(df.shape[0])
likes=[]
for i in range(df.shape[0]):
    array_split=re.split(',',df['dish_liked'][i])
    for item in array_split:
        likes.append(item)

In [ ]:
df.index=range(df.shape[0])

In [ ]:
df.index

In [ ]:
print("Count of Most liked dishes in Bangalore")
favourite_food = pd.Series(likes).value_counts()
favourite_food.head(30)

In [ ]:
ax = favourite_food.nlargest(n=20, keep='first').plot(kind='bar',figsize=(18,10),title = 'Top 30 Favourite Food counts ')

for i in ax.patches:
    ax.annotate(str(i.get_height()), (i.get_x() * 1.005, i.get_height() * 1.005))

We can infer from the analysis that the 5 most liked dishes are **Pasta**,**Pizza**,**Cocktails**,**Burgers**,and **Mocktails**

**Restaurant and their counts**

In [ ]:
plt.figure(figsize=(15,7))
rest=df['rest_type'].value_counts()[:20]
sns.barplot(rest,rest.index)
plt.title("Restaurant types")
plt.xlabel("count")

**Casual Dining**, **Quick Bites** and **Cafe** are the 3 most common types of Restaurants in Banglore

<br>
<br>

## **Most famous Restaurants**

In [ ]:
# plt.figure(figsize=(15,7))
# chains=df['name'].value_counts()[:20]
# sns.barplot(x=chains,y=chains.index,palette='Set1')
# plt.title("Most famous restaurant chains in Bangaluru",size=20,pad=20)
# plt.xlabel("Number of outlets",size=15)

<br>
<br>
<br>

# **Building Our Model**

In [ ]:
df.head()

## **Convert the online categorical variables into a numeric format**

In [ ]:
df.online_order[df.online_order == 'Yes'] = 1 
df.online_order[df.online_order == 'No'] = 0

In [ ]:
df.online_order.value_counts()

In [ ]:
df.online_order = pd.to_numeric(df.online_order)


**change the string categorical into to a categorical int**
---



In [ ]:
df.book_table[df.book_table == 'Yes'] = 1 
df.book_table[df.book_table == 'No'] = 0

In [ ]:
df.book_table = pd.to_numeric(df.book_table)

In [ ]:
df.book_table.value_counts()

Label encode the categorical variables to make it easier to build algorithm

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
df.location = le.fit_transform(df.location)
df.rest_type = le.fit_transform(df.rest_type)
df.cuisines = le.fit_transform(df.cuisines)
df.menu_item = le.fit_transform(df.menu_item)

In [ ]:
df.head()

In [55]:
my_data=df.iloc[:,[2,3,4,5,6,7,9,10,12]]
my_data.to_csv('Zomato_df.csv') 

In [56]:
x = df.iloc[:,[2,3,5,6,7,9,10,12]]
x.head()

,online_order,book_table,votes,location,rest_type,cuisines,cost,menu_item
0,1,1,775,1,20,1386,800.0,5047
1,1,0,787,1,20,594,800.0,5047
2,1,0,918,1,16,484,800.0,5047
3,0,0,88,1,62,1587,300.0,5047
4,0,0,166,4,20,1406,600.0,5047


In [57]:
y = df['rate']
y

0        4.1
1        4.1
2        3.8
3        3.7
4        3.8
        ... 
23243    3.8
23244    3.9
23245    2.8
23246    2.5
23247    4.3
Name: rate, Length: 23248, dtype: float64

In [58]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.3,random_state=10)


<br>
<br>

## **Linear Regression**

In [59]:
lr_model=LinearRegression()
lr_model.fit(x_train,y_train)

LinearRegression()

In [60]:
from sklearn.metrics import r2_score
y_pred=lr_model.predict(x_test)
r2_score(y_test,y_pred)

0.22818828522967394

<br>
<br>

## **Random Forest**

In [61]:
#from sklearn.tree import DecisionTreeRegressor

In [62]:
from sklearn.ensemble import RandomForestRegressor
RF_Model=RandomForestRegressor(n_estimators=650,random_state=245,min_samples_leaf=.0001)
RF_Model.fit(x_train,y_train)
y_predict=RF_Model.predict(x_test)
r2_score(y_test,y_predict)

0.8809706960047533

<br>
<br>

## **ExtraTree Regressor**

In [63]:
#Preparing Extra Tree Regression
from sklearn.ensemble import  ExtraTreesRegressor
ET_Model=ExtraTreesRegressor(n_estimators = 120)
ET_Model.fit(x_train,y_train)
y_predict=ET_Model.predict(x_test)


from sklearn.metrics import r2_score
r2_score(y_test,y_predict)

0.9323991902641113

Extra Tree Regressor gives us the best model

Pickle: https://bit.ly/38MGdgn

In [64]:
#Use pickle to save our model so that we can use it later

import pickle 
# Saving model to disk
pickle.dump(ET_Model, open('model.pkl','wb'))
model=pickle.load(open('model.pkl','rb'))